In [136]:

def save_file(filename, txt):
    f = open(filename, "w")
    f.write(txt)
    f.close()

In [144]:

class SPPF:
    def __init__(self, tag="", children=None):
        self.tag = tag
        self.visited = False
        self.children = children if children is not None else []
        
    def to_dict(self):
        res = {}
        res["tag"] = self.tag
        res["children"] = [ [ y.to_dict() for y in x] for x in self.children ]
        return res

class State:
    def __init__(self):
        self.LHS = ""
        self.RHS = []
        self.pos = -1; # 0 means the beginning
        self.origin = 0
        self.SPPF = SPPF()

    #debug only
    def str(self):
        res = self.LHS + " -> "
        for i, x in enumerate(self.RHS):
            if i == self.pos:
                res += ". "
            res  += x + " "
        if "." not in res:
            res += ". "
        res += "| " + str(self.origin)
        return res

    def __eq__(self, other):
        return self.LHS == other.LHS and self.RHS == other.RHS and self.pos == other.pos and self.origin == other.origin

In [145]:
def read_rules(string):
    res = {}
    for rule in string.split("\n"):
        head, body = rule.split("->")
        head = head.strip()
        body = [ x.strip() for x in body]
        body = [ x for x in body if x != ""]
        if head not in res:
            res[head] = []
        res[head].append(body)
    return res

In [146]:
def all_combs_of_2(list1, list2):
    #list1 is a list of lists, list2 is just list
    res = []
    for x in list1:
        for y in list2:
            res.append( x + [y])
    return res
    
def all_combs(lists):
    #function that takes a list L of lists l0,l1,l2... and returns of combination of their elements
    res = [ [x] for x in lists[0]]
    for x in lists[1:]:
        res = all_combs_of_2(res, x)
    return res

def unpack(tree):
    res = []
    if len(tree.children) == 0:
        res = [tree.tag]
    for alt in tree.children:
        children = [ unpack(child) for child in alt] #children is a list of lists
        combinations = all_combs(children)
        for x in combinations:
            res.append (tree.tag + "["+ " ".join(x) + "]")
    return res
        

In [147]:
import copy
class Parser:
    def __init__(self):
        self.sets = []

    def _debug(self, i):
            #debugging 
            print("-" * 30, i, "-"*30)
            for s in self.sets[i]:
                print(s.str())
                
    def parse(self, word, rules, head):
        self.sets = [ [] for x in range(len(word)+1) ]
        self.leaves = [SPPF(i) for i in word]
        #add the first state
        for RHS in rules[head]:
            state = State()
            state.LHS = head
            state.RHS = RHS
            state.pos = 0
            state.origin = 0
            state.SPPF.tag = head
            self.sets[0].append(state)

        
        for i in range(len(word)):
             
            j = 0
            while j < len(self.sets[i]):
                state = self.sets[i][j]
                if state.pos != len(state.RHS):
                    if state.RHS[state.pos] == word[i]:
                        self.scanner(state, i)
                    else:
                        self.predict(rules, state, i)
                else:
                    self.complete(state, i)
                j += 1
            
        for s in self.sets[i+1]:
            if s.pos == len(s.RHS):
                self.complete(s,i+1)
            else:
                self.predict(rules, s, i+1)

        res = []
        self.roots = []
        for s in self.sets[i+1]:
            if s.LHS == head and s.pos == len(s.RHS) and s.origin == 0:
                res.append(s)
                self.roots.append(s.SPPF)

        return res
            
    def scanner(self, state, i):
        _state = copy.deepcopy(state)
        _state.pos += 1
        leaf = self.leaves[i]
        if len(_state.SPPF.children) == 0:
            _state.SPPF.children.append([leaf])
        else:
            for child in _state.SPPF.children:
                child.append(leaf)
        
        if _state not in self.sets[i+1]:
            self.sets[i+1].append(_state)

    def predict(self, rules, state, i):
        next_nterm = state.RHS[state.pos]
        for head, RHSs in rules.items():
            if head == next_nterm:
                for RHS in RHSs:
                    _state = State()
                    _state.LHS = head
                    _state.RHS = RHS
                    _state.pos = 0
                    _state.origin = i
                    #or should we keep the old sppf?
                    _state.SPPF = SPPF(head, [])
                    if _state not in self.sets[i]:
                        self.sets[i].append(_state)
                        #no need to call for predict recurently, the new state will be noticed and predicted in the main loop

    def complete(self, state, i):
        for s in self.sets[state.origin]:
            if s.pos < len(s.RHS) and s.RHS[s.pos] == state.LHS:
                _state = copy.deepcopy(s)
                _state.pos += 1

                
                
                leaf = state.SPPF
                if len(_state.SPPF.children) == 0:
                    _state.SPPF.children.append([leaf])
                else:
                    for child in _state.SPPF.children:
                        child.append(leaf)

                if _state not in self.sets[i]:
                    self.sets[i].append(_state)
                else:
                    #we just add the alternative children
                    #print("the evil state is ccaused by: ", state.str())
                    pos = self.sets[i].index(_state)
                    self.sets[i][pos].SPPF.children.extend(_state.SPPF.children)
                
        

In [148]:
def read_sppf(parser):
    sppfs = {}
    trans = {}
    
    def normalize(node):
        node = id(node)
        if node not in trans:
            trans[node] = len(trans)
        return trans[node]
    
    leaves = [(l.tag, normalize(l)) for l in parser.leaves]
    
    
    for s in parser.sets:
        for stat in s:
            
            _id = normalize(stat.SPPF)
            
            tab1 = []
            
            for alt in stat.SPPF.children:
                tab = []
                for child in alt:
                    tab.append(normalize(child))
                
                tab1.append(tab)
            
            sppfs[_id] = (stat.SPPF.tag, tab1)
    
    roots = [ normalize(x) for x in parser.roots]
    
    for k,v in sppfs.items():
        print(k, v)
    print("leaves: ",leaves)
    print("roots: ", roots)

In [151]:
def test(rules_str, head, input):
    rules = read_rules(rules_str)
    p = Parser()
    res = p.parse(input, rules, head)
    
    read_sppf(p)
    
    res = [unpack(x.SPPF) for x in res][0]
    print (res)

In [152]:
rules = """ S -> S S 
S -> b"""
test(rules, "S", ["b","b","b"])

3 ('S', [])
4 ('S', [])
5 ('S', [[0]])
6 ('S', [[5]])
7 ('S', [])
8 ('S', [])
9 ('S', [[1]])
10 ('S', [[11, 9]])
12 ('S', [[9]])
13 ('S', [[10]])
14 ('S', [])
15 ('S', [])
16 ('S', [[2]])
17 ('S', [[18, 16]])
19 ('S', [[20, 16], [21, 17]])
22 ('S', [[16]])
23 ('S', [[17]])
24 ('S', [[19]])
25 ('S', [])
26 ('S', [])
leaves:  [('b', 0), ('b', 1), ('b', 2)]
roots:  [19]
['S[S[S[b] S[b]] S[b]]', 'S[S[b] S[S[b] S[b]]]']


In [153]:
input = ["x", "+", "x", "*", "x"]
rules = """ P -> S
S -> S + M
S -> M
M -> M * T
M -> T
T -> x """
test(rules, "P", input)

5 ('P', [])
6 ('S', [])
7 ('S', [])
8 ('M', [])
9 ('M', [])
10 ('T', [])
11 ('T', [[0]])
12 ('M', [[11]])
13 ('S', [[12]])
14 ('M', [[12]])
15 ('P', [[13]])
16 ('S', [[13]])
17 ('S', [[18, 1]])
19 ('M', [])
20 ('M', [])
21 ('T', [])
22 ('T', [[2]])
23 ('M', [[22]])
24 ('S', [[25, 26, 23]])
27 ('M', [[23]])
28 ('P', [[24]])
29 ('S', [[24]])
30 ('M', [[31, 3]])
32 ('T', [])
33 ('T', [[4]])
34 ('M', [[35, 36, 33]])
37 ('S', [[38, 39, 34]])
40 ('M', [[34]])
41 ('P', [[37]])
42 ('S', [[37]])
leaves:  [('x', 0), ('+', 1), ('x', 2), ('*', 3), ('x', 4)]
roots:  [41]
['P[S[S[M[T[x]]] + M[M[T[x]] * T[x]]]]']


In [154]:
rules = """ S -> a S
S -> a """
input = ["a","a","a","a"]
test(rules, "S", input)

4 ('S', [])
5 ('S', [])
6 ('S', [[0]])
7 ('S', [[0]])
8 ('S', [])
9 ('S', [])
10 ('S', [[1]])
11 ('S', [[1]])
12 ('S', [])
13 ('S', [])
14 ('S', [[15, 11]])
16 ('S', [[2]])
17 ('S', [[2]])
18 ('S', [])
19 ('S', [])
20 ('S', [[21, 17]])
22 ('S', [[23, 20]])
24 ('S', [[3]])
25 ('S', [[3]])
26 ('S', [])
27 ('S', [])
28 ('S', [[29, 25]])
30 ('S', [[31, 28]])
32 ('S', [[33, 30]])
leaves:  [('a', 0), ('a', 1), ('a', 2), ('a', 3)]
roots:  [32]
['S[a S[a S[a S[a]]]]']


In [155]:
rules = """ S ->  S a
S -> a """
input = ["a","a","a","a"]
test(rules, "S", input)

4 ('S', [])
5 ('S', [])
6 ('S', [[0]])
7 ('S', [[6]])
8 ('S', [[9, 1]])
10 ('S', [[8]])
11 ('S', [[12, 2]])
13 ('S', [[11]])
14 ('S', [[15, 3]])
16 ('S', [[14]])
leaves:  [('a', 0), ('a', 1), ('a', 2), ('a', 3)]
roots:  [14]
['S[S[S[S[a] a] a] a]']


In [156]:
rules = """ S ->  S A
S -> A
A -> a 
A -> a a"""
input = ["a","a","a","a"]
test(rules, "S", input)

4 ('S', [])
5 ('S', [])
6 ('A', [])
7 ('A', [])
8 ('A', [[0]])
9 ('A', [[0]])
10 ('S', [[8]])
11 ('S', [[10]])
12 ('A', [])
13 ('A', [])
14 ('A', [[15, 1]])
16 ('A', [[1]])
17 ('A', [[1]])
18 ('S', [[14]])
19 ('S', [[20, 16]])
21 ('S', [[18], [19]])
22 ('A', [])
23 ('A', [])
24 ('A', [[25, 2]])
26 ('A', [[2]])
27 ('A', [[2]])
28 ('S', [[29, 24], [30, 26], [31, 26]])
32 ('S', [[28]])
33 ('A', [])
34 ('A', [])
35 ('A', [[36, 3]])
37 ('A', [[3]])
38 ('A', [[3]])
39 ('S', [[40, 35], [41, 35], [42, 37]])
43 ('S', [[39]])
44 ('A', [])
45 ('A', [])
leaves:  [('a', 0), ('a', 1), ('a', 2), ('a', 3)]
roots:  [39]
['S[S[A[a a]] A[a a]]', 'S[S[S[A[a]] A[a]] A[a a]]', 'S[S[S[A[a]] A[a a]] A[a]]', 'S[S[S[A[a a]] A[a]] A[a]]', 'S[S[S[S[A[a]] A[a]] A[a]] A[a]]']


In [157]:
rules = """ S ->  a S B B
S -> a
B -> b
B -> """
input = ["a","a","a","b"]
test(rules, "S", input)

4 ('S', [])
5 ('S', [])
6 ('S', [[0]])
7 ('S', [[0]])
8 ('S', [])
9 ('S', [])
10 ('S', [[1]])
11 ('S', [[1]])
12 ('S', [])
13 ('S', [])
14 ('S', [[15, 11]])
16 ('B', [])
17 ('B', [])
18 ('S', [[19, 20, 17]])
21 ('S', [[22, 23, 24, 17]])
25 ('S', [[2]])
26 ('S', [[2]])
27 ('S', [])
28 ('S', [])
29 ('S', [[30, 26]])
31 ('B', [])
32 ('B', [])
33 ('S', [[34, 35, 32]])
36 ('S', [[37, 38, 39, 32]])
40 ('S', [[41, 36]])
42 ('B', [[3]])
43 ('S', [[44, 45, 42]])
46 ('S', [[47, 48, 49, 42], [50, 51, 52, 53]])
54 ('S', [[55, 56, 42], [57, 58, 53]])
59 ('B', [])
53 ('B', [])
60 ('S', [[61, 46]])
62 ('S', [[63, 64, 65, 53]])
leaves:  [('a', 0), ('a', 1), ('a', 2), ('b', 3)]
roots:  [62]
['S[a S[a S[a] B B] B[b] B]']


In [97]:
rules = """S -> A
A -> B
B -> A
B -> a """
input = ["a"]
test(rules, "S", input)

nodes:  {1: [], 2: [], 3: [], 4: [], 5: [[0]], 6: [[5], [7]], 8: [[6]], 7: [[6]]}
leaves:  [0]


In [30]:
str(res1)

'["[\'S[S[S[b] S[b]] S[b]]\', \'S[S[b] S[S[b] S[b]]]\']", "[\'P[S[S[M[T[x]]] + M[M[T[x]] * T[x]]]]\']", "[\'S[a S[a S[a S[a]]]]\']", "[\'S[S[S[S[a] a] a] a]\']", "[\'S[S[A[a a]] A[a a]]\', \'S[S[S[A[a]] A[a]] A[a a]]\', \'S[S[S[A[a]] A[a a]] A[a]]\', \'S[S[S[A[a a]] A[a]] A[a]]\', \'S[S[S[S[A[a]] A[a]] A[a]] A[a]]\']", "[\'S[a S[a S[a] B B] B[b] B]\']"]'

In [31]:
str(res2) ==str(res1)

True

In [58]:
res[0].SPPF.children[0][0].children[1][0].children[0][0].children

[[<__main__.SPPF at 0x266c0d62e10>], [<__main__.SPPF at 0x266bf078650>]]